    Here seconds tick data is converted to per minute OHLC

In [125]:
import pandas as pd
import numpy as np
from datetime import date

In [126]:
# Here I have kept tick data in the dataframe
path = '/home/debashis/Data/Finance_Data/Live_data_work/scripts/Tick_data3/'
df_nifty_bank = pd.read_csv(path+'Nifty_26_march.csv', low_memory=False)
df_nifty_bank.head()

,timestamp,ltp,volume
0,2021-03-26 09:15:49,14487.70,0
1,2021-03-26 09:15:50,14489.60,0
2,2021-03-26 09:15:51,14490.35,0
3,2021-03-26 09:15:52,14492.20,0
4,2021-03-26 09:15:53,14493.95,0


In [127]:
# timestamp is treated as date_time here
df_nifty_bank['timestamp'] = pd.to_datetime(df_nifty_bank['timestamp'])
df_nifty_bank.head()

,timestamp,ltp,volume
0,2021-03-26 09:15:49,14487.70,0
1,2021-03-26 09:15:50,14489.60,0
2,2021-03-26 09:15:51,14490.35,0
3,2021-03-26 09:15:52,14492.20,0
4,2021-03-26 09:15:53,14493.95,0


In [128]:
# df_nifty_bank.timestamp.values[0]

    Task:
         - Present data in each seconds
         - Generate OHLC for each minute
         

In [129]:
# Returns the current local date
# today = date.today()
today = '2021-03-26'
print("Today date is: ", today)

Today date is:  2021-03-26


In [130]:
start_time = str(today)+' 09:15:00'
end_time = str(today)+' 15:29:00'
print('Start time: {0}, End time: {1}'.format(start_time, end_time))

Start time: 2021-03-26 09:15:00, End time: 2021-03-26 15:29:00


In [131]:
# create minute-wise timestamp
time_bin_1_min = pd.date_range(start=start_time, end = end_time, freq='1min')
print('Length of Minutes data: ', len(time_bin_1_min))

Length of Minutes data:  375


In [132]:
# Create the data frame
required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
df_minute_data = pd.DataFrame(columns=required_columns)
df_minute_data.head()

# Create empty lists to store data
date, open_, high, low, close, volume = [], [], [], [], [], []

for x in range(len(time_bin_1_min)-1):
    # Collect all LTP data for this minute
    start_minute = time_bin_1_min[x]
    end_minute = time_bin_1_min[x+1]
#     print('Time Range: {0} to {1}'.format(start_minute, end_minute))
    
    # Create the mask
    mask = ((df_nifty_bank['timestamp'] >= start_minute) & (df_nifty_bank['timestamp'] < end_minute))
    
    df_temp = df_nifty_bank.loc[mask]
    temp_open = df_temp.ltp.values[0]
    temp_low = min(df_temp.ltp.values)
    temp_high = max(df_temp.ltp.values)
    temp_close = df_temp.ltp.values[-1]
    temp_volume = np.mean(df_temp.volume.values)
#     print('Open, High, Low, Close: ', temp_open, temp_high, temp_low, temp_close)
    date.append(start_minute)
    open_.append(temp_open)
    high.append(temp_high)
    low.append(temp_low)
    close.append(temp_close)
    volume.append(temp_volume)
    
df_minute_data.Date = date
df_minute_data.Open = open_
df_minute_data.High = high
df_minute_data.Low = low
df_minute_data.Close = close
df_minute_data.Volume = volume

In [133]:
df_minute_data.describe()

,Open,High,Low,Close,Volume
count,374.000000,374.00000,374.000000,374.000000,374.0
mean,14508.464973,14513.25762,14503.904144,14508.375267,0.0
std,33.371092,32.47818,33.687727,33.204302,0.0
min,14419.850000,14426.55000,14415.350000,14420.150000,0.0
25%,14482.612500,14486.02500,14478.575000,14482.637500,0.0
50%,14518.725000,14523.07500,14514.600000,14517.775000,0.0
75%,14534.987500,14538.43750,14529.975000,14535.000000,0.0
max,14566.550000,14572.65000,14562.950000,14567.200000,0.0


In [134]:
df_minute_data.head()

,Date,Open,High,Low,Close,Volume
0,2021-03-26 09:15:00,14487.70,14496.2,14487.70,14493.25,0.0
1,2021-03-26 09:16:00,14494.10,14494.1,14461.25,14468.90,0.0
2,2021-03-26 09:17:00,14468.40,14474.7,14458.65,14464.30,0.0
3,2021-03-26 09:18:00,14464.60,14487.7,14464.60,14482.75,0.0
4,2021-03-26 09:19:00,14481.25,14483.2,14461.45,14462.55,0.0


In [135]:
df_minute_data.tail()

,Date,Open,High,Low,Close,Volume
369,2021-03-26 15:24:00,14524.45,14530.55,14520.40,14522.65,0.0
370,2021-03-26 15:25:00,14523.00,14528.10,14520.75,14528.10,0.0
371,2021-03-26 15:26:00,14528.35,14537.30,14526.90,14530.35,0.0
372,2021-03-26 15:27:00,14532.50,14532.85,14526.50,14528.10,0.0
373,2021-03-26 15:28:00,14528.70,14530.05,14522.70,14524.45,0.0


In [137]:
# Save per minute data to disk

# df_minute_data.to_csv('/home/debashis/Data/Finance_Data/Live_data_work/Minutes_data/'+str(today)+'_per_minutes_NIFTY_.csv',
#                       index=False)